[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pronobis/libspn-keras/blob/master/examples/notebooks/Understanding%20Region%20SPNs.ipynb)

In [ ]:
!pip install libspn-keras

# Understanding Region SPNs
In this notebook will have a closer look at how `libspn-keras` accomplishes scalable, layer-wise implementations of SPNs with arbitrary _decompositions_. 

### Decompositions
A decomposition is a way of groupig a set of random variables hierarchically, which is also what SPNs do. 

### Regions
In SPN literature, a decomposition is sometimes referred to as a _region graph_. Each region in such a graph corresponds to a single scope. SPNs need to be decomposable and complete, so as soon as two regions need to 'combine' their scopes, they can only do so if their scopes have no overlap at all. Within a single region, one can use sum nodes without breaking completeness or decomposability, hence maintaining the validity of the SPN.

## Regions in `libspn-keras`
In `libspn-keras`, region graphs can be built using `RegionNode` and `RegionVariable`. In a region graph there is no explicit notion of products or sums, those will be dealt with later. The region graphs are simply used to define the scope structure. Nothing more, nothing less.

Imagine we have three random variables $X_0$, $X_1$ and $X_2$. There are several ways of grouping these variables into regions. The smallest possible region is one that consists of a single variable.

In [2]:
import libspn_keras as spnk

x0 = spnk.RegionVariable(0)
x1 = spnk.RegionVariable(1)
x2 = spnk.RegionVariable(2)

print(x0)
print(x1)
print(x2)

x0
x1
x2


### Combining regions
Two regions can be combined if their scopes don't overlap. We can combine the regions we have above in several ways: 

In [3]:
x0_x1 = spnk.RegionNode([x0, x1])
x0_x2 = spnk.RegionNode([x0, x2])
x1_x2 = spnk.RegionNode([x1, x2])
x0_x1_x2 = spnk.RegionNode([x0, x1, x2])

print(x0_x1)
print(x0_x2)
print(x1_x2)
print(x0_x1_x2)

{x0, x1}
{x0, x2}
{x1, x2}
{x0, x1, x2}


### Scope overlap
As stated before, one thing we're not allowed to do is combine regions whose scopes overlap. In other words, the intersection of the variable sets that any pair of combined regions have must be empty. Here are a few examples of things we're not allowed to do:

In [4]:
def try_to_combine(a, b):
    try:
        spnk.RegionNode([a, b])
    except spnk.region.OverlappingScopesException as e:
        print(e)

try_to_combine(x0, x0)
try_to_combine(x0, x0_x1)
try_to_combine(x0_x1, x1_x2)

Children x0 and x0 have overlapping scopes
Children x0 and {x0, x1} have overlapping scopes
Children {x0, x1} and {x1, x2} have overlapping scopes


### Root regions
A region can be considered a _root region_ if it covers all random variables. So far, we've only made one root region `x0_x1_x2`. A _root region_ cannot be combined with any other region anymore as there would always be scope overlap. 

Here are some other root regions:

In [5]:
x0x1_x2 = spnk.RegionNode([x0_x1, x2])
x0x2_x1 = spnk.RegionNode([x0_x2, x1])
x1x2_x0 = spnk.RegionNode([x1_x2, x0])

print(x0x1_x2)
print(x0x2_x1)
print(x1x2_x0)

{{x0, x1}, x2}
{{x0, x2}, x1}
{{x1, x2}, x0}


## From region graphs to SPNs
Now that we have the region graphs, we can compose stacks of product and sum layers that adhere to their hierarchical scope structure. By using `region_graph_to_dense_spn` we can easily convert a region graph to an SPN. 

We also have to specify the leaf layer and the number of sums at each depth of the SPN. Note that for the root of the SPN, we will only have a single sum. Therefore, we don't have to specify a `1` in our `num_sums_iterable`.

In [6]:
# Number of sums per region from bottom to top
num_sums_iterable = iter([2, 2])

leaf = spnk.layers.NormalLeaf(num_components=2)
dense_spn = spnk.region_graph_to_dense_spn(
    x0x2_x1, 
    num_sums_iterable=num_sums_iterable,
    leaf_node=leaf,
    return_weighted_child_logits=False
)

dense_spn.summary(line_length=80)

Model: "sequential"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
flat_to_regions (FlatToRegions)     (None, 3, 1, 1)                 0           
________________________________________________________________________________
normal_leaf (NormalLeaf)            (None, 3, 1, 2)                 12          
________________________________________________________________________________
permute_and_pad_scopes (PermuteAndP (None, 4, 1, 2)                 0           
________________________________________________________________________________
dense_product (DenseProduct)        (None, 2, 1, 4)                 0           
________________________________________________________________________________
dense_sum (DenseSum)                (None, 2, 1, 2)                 16          
________________________________________________________________________________
dense_pr

### The layers in a dense SPN
As you can see in the summary above, we have a variety of layer types in our SPN. The first layer that is inserted is a `FlatToRegion` layer. This layer takes in a tensor with shape `[num_batch, num_variables, variable_dimensionality]` and produces an output of `[num_variables, num_decompositions, num_batch, variable_dimensionality]`. 

Currently, `region_graph_to_dense_spn` always converts a region to an SPN with one decomposition. Also, it assumes the input variables have a dimensionality of 1 (they are univariate). Hence, the output shape of the first layer is `(3, 1, None, 1)`. 

Then, we apply the leaf distributions in `NormalLeaf`. This is where the raw observations get transformed to 'probabilities'. 

The `PermuteAndPadScopes` layer permutes the scopes so that subsequent product layers can combine scopes simply by multiplying _adjacent_ scopes. In this case, the first product layer computes the product for the regions `x0` and `x1` simply by multiplying all values in `permute_and_pad_scopes_out[0, ...]` with `permute_and_pad_scopes_out[1, ...]` and putting that into `dense_product_out[0, ...]`. 

The second set of products ends up in `dense_product_out[1, ...]` and computes all products between the nodes in `permute_and_pad_scopes_out[2, ...]` and `permute_and_pad_scopes_out[3, ...]`. This can be implemented with just a handful of operations in `tensorflow`.

### Padding

Optimal GPU parallelization is the motivation behind the **padding** in `PermuteAndPadScopes`. By padding with 'dummy' nodes with empty scopes and a fixed probability of 1 (or equivalently 0 in log-space), we obtain a tree structure that is fully balanced and homogeneous. In other words, all nodes in a single layer have the same number of children, thus allowing us to 'broadcast' the internal operations of the layers across all regions, decompositions and batch samples!  

### Visualizing
For educational purposes, `libspn-keras` implements a `visualize_dense_spn` function that visualizes the structure of the SPN using `plotly`:

In [8]:
spnk.visualize_dense_spn(dense_spn, show_legend=True, transparent=False).show()

You might need to resize the window to make the legend appear in the plot. You can see that sibling nodes with the same scope are grouped by their color. The `*` nodes are the padded nodes that we need to insert to make the tree perfectly balanced. 

Notice how the ``PermuteAndPadScopes`` layer permutes the very first layer and pads it with two more nodes on the right.

The probability of the padded nodes is fixed to 1 and so the above network is equivalent to:


In [ ]:
spnk.visualize_dense_spn(dense_spn, show_legend=True, show_padding=False, transparent=False).show()

Of course, if there is no need for padding, then we shouldn't. In the next snippet we have an SPN where we don't need padding to get a perfectly balanced tree.

In [ ]:
# Number of sums per region from bottom to top
num_sums_iterable = iter([])  # in this case there are no non-root sum layers

leaf = spnk.layers.NormalLeaf(num_components=2)
dense_spn = spnk.region_graph_to_dense_spn(
    x0_x1_x2, 
    num_sums_iterable=num_sums_iterable,
    leaf_node=leaf,
    return_weighted_child_logits=False
)
spnk.visualize_dense_spn(dense_spn, show_legend=True, transparent=False)

### Large SPNs
In practice, you might want to make things larger, for example:

In [ ]:
import itertools

nodes = [spnk.RegionVariable(i) for i in range(32)]

while len(nodes) > 1:
    next_nodes = []
    for i in range(0, len(nodes), 2):
        next_nodes.append(spnk.RegionNode([nodes[i], nodes[i + 1]]))
    nodes = next_nodes

leaf = spnk.layers.NormalLeaf(num_components=2)
dense_spn = spnk.region_graph_to_dense_spn(
    nodes[0], 
    num_sums_iterable=itertools.cycle([2]),
    leaf_node=leaf,
    return_weighted_child_logits=False
) 
spnk.visualize_dense_spn(dense_spn, node_size=16, transparent=False)